In [8]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [9]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4090,6.7,0.240,0.30,10.20,0.070,44.0,179.0,0.99666,2.86,0.46,8.9,6
2970,8.4,0.310,0.31,0.95,0.021,52.0,148.0,0.99038,2.93,0.32,11.5,5
4853,5.2,0.250,0.23,1.40,0.047,20.0,77.0,0.99001,3.32,0.62,11.4,5
97,8.6,0.265,0.36,1.20,0.034,15.0,80.0,0.99130,2.95,0.36,11.4,7
225,7.1,0.240,0.41,17.80,0.046,39.0,145.0,0.99980,3.32,0.39,8.7,5
480,6.7,0.470,0.34,8.90,0.043,31.0,172.0,0.99640,3.22,0.60,9.2,5
1461,6.4,0.145,0.49,5.40,0.048,54.0,164.0,0.99460,3.56,0.44,10.8,6
704,5.6,0.250,0.26,3.60,0.037,18.0,115.0,0.99040,3.42,0.50,12.6,6
4135,5.9,0.260,0.24,2.40,0.046,27.0,132.0,0.99234,3.63,0.73,11.3,5
4856,7.1,0.230,0.39,13.70,0.058,26.0,172.0,0.99755,2.90,0.46,9.0,6


In [10]:
remote_server_uri = "http://0.0.0.0:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [11]:
mlflow.tracking.get_tracking_uri()

'http://0.0.0.0:5000'

In [12]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2024/11/26 08:04:27 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/premkumargajji/pydataberlin-2019/mlruns/891091769369979184', creation_time=1732608267620, experiment_id='891091769369979184', last_update_time=1732608267620, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [14]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [8]:
# could also do
# with mlflow.start_run():
#     for epoch in range(0, 3):
#          mlflow.log_metric(key="quality", value=2*epoch, step=epoch)

In [15]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772689
Save to: /home/premkumargajji/pydataberlin-2019/mlruns/891091769369979184/f18fbbd271da47109d9168c92298e78a/artifacts


2024/11/26 08:04:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run calm-gnu-987 at: http://0.0.0.0:5000/#/experiments/891091769369979184/runs/f18fbbd271da47109d9168c92298e78a
🧪 View experiment at: http://0.0.0.0:5000/#/experiments/891091769369979184


In [16]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062341
  MAE: 0.6155290394093893
  R2: 0.20224631822892103
Save to: /home/premkumargajji/pydataberlin-2019/mlruns/891091769369979184/8ffd60bd3abb427f8bd578680a7dc98e/artifacts


2024/11/26 08:05:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run unequaled-foal-396 at: http://0.0.0.0:5000/#/experiments/891091769369979184/runs/8ffd60bd3abb427f8bd578680a7dc98e
🧪 View experiment at: http://0.0.0.0:5000/#/experiments/891091769369979184


In [17]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: /home/premkumargajji/pydataberlin-2019/mlruns/891091769369979184/b127a77d7e934177acec0e540d1e0403/artifacts


2024/11/26 08:05:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run resilient-grub-229 at: http://0.0.0.0:5000/#/experiments/891091769369979184/runs/b127a77d7e934177acec0e540d1e0403
🧪 View experiment at: http://0.0.0.0:5000/#/experiments/891091769369979184


In [18]:
from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.search_experiments() # returns a list of mlflow.entities.Experiment
print(experiments)

[<Experiment: artifact_location='/home/premkumargajji/pydataberlin-2019/mlruns/891091769369979184', creation_time=1732608267620, experiment_id='891091769369979184', last_update_time=1732608267620, lifecycle_stage='active', name='ElasticNet_wine', tags={}>]


In [20]:
# get the run
_run = client.get_run(run_id="b127a77d7e934177acec0e540d1e0403")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6112547988118587,
 'r2': 0.2157063843066196,
 'rmse': 0.7792546522251949}, params={'alpha': '0.1', 'l1_ratio': '0.1'}, tags={'mlflow.log-model.history': '[{"run_id": "b127a77d7e934177acec0e540d1e0403", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2024-11-26 08:05:03.661726", "model_uuid": '
                             '"db169a5b0da541a7941492e4d7005b51", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.13.0", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.5.2", '
                            